# DJANGO MAS JUPYTER NOTEBOOK

TESTING ENDPOINT

In [1]:
import pandas as pd
import requests
import json
import random
from django_for_jupyter import init_django

In [2]:
init_django("django_project")

In [3]:
from rest_application.models import Profile

Profile.objects.all()[:10]

<QuerySet [<Profile: Alejandra Bordo, (20464260)>, <Profile: Pedro Rojo, (10217811)>, <Profile: Carola Naranja, (8163234)>, <Profile: Mario Amarillo, (28219272)>, <Profile: Carola Bordo, (16250004)>, <Profile: Sofia Naranja, (8849573)>, <Profile: Carola Amarillo, (9920210)>, <Profile: Mario Bordo, (52650007)>, <Profile: Jorge Naranja, (10692008)>, <Profile: Victoria Rojo, (55650009)>]>

In [4]:
def get_pandas_profiles(model):
    data = model.objects.all().values()
    df = pd.DataFrame(data)
    return df

In [5]:
get_pandas_profiles(Profile)

,personal_id,name,last_name,age
0,20464260,Alejandra,Bordo,60
1,10217811,Pedro,Rojo,73
2,8163234,Carola,Naranja,88
3,28219272,Mario,Amarillo,39
4,16250004,Carola,Bordo,65
...,...,...,...,...
2013,55555555,development,Amarillo,40
2014,113,Federico,Gris,22
2015,55555556,developer,Amarillo,30
2016,55555557,Daddy,Copenaghen,50


# PRIMER TEST: api/v1/profiles/

Metodo GET: <br />
    - Test 1 | Sin query_params: Devuelve todos los perfiles paginados<br />
    - Test 2 | Con query_params: Devuelve los perfiles que cumplan con los query_params<br />
    - Test 3 | Con query_params que no existen: Devuelve un error 404<br />
    - Test 4 | Con query_params que no son validos: Devuelve un error 400<br />
<br />
Metodo POST: <br />
    - Test 1 | Con data: Devuelve el perfil creado<br />
    - Test 2 | Sin data: Devuelve un error 400<br />
    - Test 3 | Con data que no es valida: Devuelve un error 400<br />


In [6]:
# test 1 GET: sin query_params

response = requests.request(method="GET", url="http://localhost:8000/api/v1/profiles/")

print(response.content)
print("longitud:", len(json.loads(response.content)))

b'{"count":2018,"next":"http://localhost:8000/api/v1/profiles/?page=2","previous":null,"results":[{"personal_id":"20464260","name":"Alejandra","last_name":"Bordo","age":60},{"personal_id":"10217811","name":"Pedro","last_name":"Rojo","age":73},{"personal_id":"8163234","name":"Carola","last_name":"Naranja","age":88},{"personal_id":"28219272","name":"Mario","last_name":"Amarillo","age":39},{"personal_id":"16250004","name":"Carola","last_name":"Bordo","age":65}]}'
longitud: 4


In [7]:
# test 2 GET: con query_params válidos
name = "Agustin"
last_name = ""
age = ""

url = "http://localhost:8000/api/v1/profiles/"
query = f"?name={name}&last_name={last_name}&age={age}"

response = requests.request(method="GET", url=url + query)

print("status_code:", response.status_code)
print("content:\n", json.loads(response.content))

status_code: 200
content:
 {'count': 210, 'next': 'http://localhost:8000/api/v1/profiles/?age=&last_name=&name=Agustin&page=2', 'previous': None, 'results': [{'personal_id': '24319292', 'name': 'Agustin', 'last_name': 'Amarillo', 'age': 54}, {'personal_id': '8163247', 'name': 'Agustin', 'last_name': 'Rojo', 'age': 88}, {'personal_id': '22555724', 'name': 'Agustin', 'last_name': 'Violeta', 'age': 57}, {'personal_id': '27419295', 'name': 'Agustin', 'last_name': 'Bordo', 'age': 46}, {'personal_id': '9565990', 'name': 'Agustin', 'last_name': 'Bordo', 'age': 79}]}


In [8]:
# test 3 GET: con query_params inexistentes

name = ""
last_name = ""
age = "1"

url = "http://localhost:8000/api/v1/profiles/"
query = f"?name={name}&last_name={last_name}&age={age}"

response = requests.request(method="GET", url=url + query)

try:
    data = json.loads(response.content)
    print("status_code:", response.status_code)
    print("content:\n", response.content)
    print("longitud:", len(data))
except:
    print("status_code:", response.status_code)

status_code: 404


In [9]:
# test 4 GET: con query_params inválidos

name = ""
last_name = ""
age = "anything"

url = "http://localhost:8000/api/v1/profiles/"
query = f"?name={name}&last_name={last_name}&age={age}"

response = requests.request(method="GET", url=url + query)

try:
    data = json.loads(response.content)
    print("status_code:", response.status_code)
    print("content:\n", response.content)
    print("longitud:", len(data))
except:
    print("status_code:", response.status_code)

status_code: 400


In [10]:
# test 1 POST: crear un registro

status = 1

while status != 201:

    new_id = random.randint(2, 1000)

    data = {"personal_id": new_id, "name": "Federico", "last_name": "Gris", "age": 22}

    response = requests.request(
        method="POST", url="http://localhost:8000/api/v1/profiles/", data=data
    )

    status = response.status_code

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b'{"personal_id":"475","name":"Federico","last_name":"Gris","age":22}'
status_code: 201


In [11]:
# test 2 POST: sin data

data = {}

response = requests.request(
    method="POST", url="http://localhost:8000/api/v1/profiles/", data=data
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b'{"personal_id":["This field is required."],"name":["This field is required."],"last_name":["This field is required."],"age":["This field is required."]}'
status_code: 400


In [12]:
# test 3 POST: con data inválida
data = {"personal_id": "ABC", "name": "123", "last_name": "Verde", "age": "all"}

response = requests.request(
    method="POST", url="http://localhost:8000/api/v1/profiles/", data=data
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b'{"personal_id":["Personal ID must be a number, not ABC"],"name":["Name and Last Name must be a string, not 123"],"age":["A valid integer is required."]}'
status_code: 400


# SEGUNDO TEST: api/v1/profiles/\<pk\>/

Metodo GET: <br />
    - Test 1 | Con pk que existe: Devuelve el perfil<br />
    - Test 2 | Con pk que no es valido: Devuelve un error 400<br />
    - test 3 | Con pk que no existe: Devuelve un error 404<br />
<br />
Metodo PUT: <br />
    - Test 1 | Con pk que existe: Devuelve el perfil actualizado<br />
    - Test 2 | Con pk que no es valido: Devuelve un error 400<br />
    - Test 3 | Con pk que no existe: Devuelve un error 404<br />
    - Test 4 | Con data que no es valida: Devuelve un error 400<br />
    - Test 5 | Sin data: Devuelve un error 400<br />
<br />
Metodo DELETE: <br />
    - Test 1 | Con pk que existe: Devuelve un status 204<br />
    - Test 2 | Con pk que no existe: Devuelve un error 404<br />
    - Test 3 | Con pk que no es valido: Devuelve un error 400<br />

In [13]:
# test 1 GET: con pk válido

pk = new_id

response = requests.request(
    method="GET", url=f"http://localhost:8000/api/v1/profiles/{pk}/"
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b'{"personal_id":"475","name":"Federico","last_name":"Gris","age":22}'
status_code: 200


In [14]:
# test 2 GET: con pk inválido

pk = "ABC"

response = requests.request(
    method="GET", url=f"http://localhost:8000/api/v1/profiles/{pk}/"
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b''
status_code: 400


In [15]:
# test 3 GET: con pk inexistente

pk = 1001

response = requests.request(
    method="GET", url=f"http://localhost:8000/api/v1/profiles/{pk}/"
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b''
status_code: 404


In [16]:
# test 1 PUT: con pk válido

pk = new_id

data = {"personal_id": "1", "name": "Federico", "last_name": "Gris", "age": 22}

response = requests.request(
    method="PUT", url=f"http://localhost:8000/api/v1/profiles/{pk}/", data=data
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b'{"personal_id":"1","name":"Federico","last_name":"Gris","age":22}'
status_code: 200


In [17]:
# test 2 PUT: con pk inválido

pk = "ABC"

data = {"personal_id": "44655766", "name": "Federico", "last_name": "Gris", "age": 22}

response = requests.request(
    method="PUT", url=f"http://localhost:8000/api/v1/profiles/{pk}/", data=data
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b''
status_code: 400


In [18]:
# test 3 PUT: con pk inexistente

pk = 1001

data = {"personal_id": "44655766", "name": "Federico", "last_name": "Gris", "age": 22}

response = requests.request(
    method="PUT", url=f"http://localhost:8000/api/v1/profiles/{pk}/", data=data
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b''
status_code: 404


In [19]:
# test 4 PUT: con data inválida

pk = 1

data = {"personal_id": "ABC", "name": "123", "last_name": "132", "age": "all"}

response = requests.request(
    method="PUT", url=f"http://localhost:8000/api/v1/profiles/{pk}/", data=data
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b'{"personal_id":["Personal ID must be a number, not ABC"],"name":["Name and Last Name must be a string, not 123"],"last_name":["Name and Last Name must be a string, not 132"],"age":["A valid integer is required."]}'
status_code: 400


In [20]:
# test 5 PUT: sin data

pk = 1

data = {}

response = requests.request(
    method="PUT", url=f"http://localhost:8000/api/v1/profiles/{pk}/", data=data
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b'{"personal_id":["This field is required."],"name":["This field is required."],"last_name":["This field is required."],"age":["This field is required."]}'
status_code: 400


In [21]:
# test 1 DELETE: con pk válido

pk = 1

response = requests.request(
    method="DELETE", url=f"http://localhost:8000/api/v1/profiles/{pk}/"
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b''
status_code: 204


In [22]:
# test 2 DELETE: con pk inexistente

pk = 1001

response = requests.request(
    method="DELETE", url=f"http://localhost:8000/api/v1/profiles/{pk}/"
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b''
status_code: 404


In [23]:
# test 3 DELETE: con pk inválido

pk = "ABC"

response = requests.request(
    method="DELETE", url=f"http://localhost:8000/api/v1/profiles/{pk}/"
)

print("content:\n", response.content)
print("status_code:", response.status_code)

content:
 b''
status_code: 400
